# Running membership inference attacks on a Regression Model

In this tutorial we will show how to run black-box membership attacks. This will be demonstrated on the Diabetes dataset from Scikitlearn (https://scikit-learn.org/stable/datasets/toy_dataset.html#diabetes-dataset). 

## Load data

In [74]:
import numpy as np
import os
import sys
sys.path.insert(0, os.path.abspath('..'))

from art.utils import load_diabetes

(x_train, y_train), (x_test, y_test), _, _ = load_diabetes(test_set=0.5)

## Train MLP regression model

In [75]:
from sklearn.linear_model import LinearRegression
from art.estimators.regression.scikitlearn import ScikitlearnRegressor

model = LinearRegression()
model.fit(x_train, y_train)

art_regressor = ScikitlearnRegressor(model)

print('Base model score: ', model.score(x_test, y_test))

Base model score:  0.5182246496026136


## Attack
### Black-box attack
The black-box attack basically trains an additional classifier (called the attack model) to predict the membership status of a sample. The loss of the target model is used as input to the learning process.

In [76]:
from art.attacks.inference.membership_inference import MembershipInferenceBlackBox


bb_attack = MembershipInferenceBlackBox(art_regressor, attack_model_type='rf', input_type='loss')

attack_train_ratio = 0.5
attack_train_size = int(len(x_train) * attack_train_ratio)
attack_test_size = int(len(x_test) * attack_train_ratio)


# train attack model
bb_attack.fit(x_train[:attack_train_size], y_train[:attack_train_size],
              x_test[:attack_test_size], y_test[:attack_test_size])

# infer 
inferred_train_bb = bb_attack.infer(x_train.astype(np.float32), y_train)
inferred_test_bb = bb_attack.infer(x_test.astype(np.float32), y_test)

# check accuracy
train_acc_bb = np.sum(inferred_train_bb) / len(inferred_train_bb)
test_acc_bb = 1 - (np.sum(inferred_test_bb) / len(inferred_test_bb))
acc_bb = (train_acc_bb * len(inferred_train_bb) + test_acc_bb * len(inferred_test_bb)) / (len(inferred_train_bb) + len(inferred_test_bb))
print('Member accuracy:', train_acc_bb)
print('Non-Member accuracy:', test_acc_bb)
print('Accuracy:', acc_bb)

Member accuracy: 0.7239819004524887
Non-Member accuracy: 0.7963800904977376
Accuracy: 0.7601809954751131


In [77]:
def calc_precision_recall(predicted, actual, positive_value=1):
    score = 0  # both predicted and actual are positive
    num_positive_predicted = 0  # predicted positive
    num_positive_actual = 0  # actual positive
    for i in range(len(predicted)):
        if predicted[i] == positive_value:
            num_positive_predicted += 1
        if actual[i] == positive_value:
            num_positive_actual += 1
        if predicted[i] == actual[i]:
            if predicted[i] == positive_value:
                score += 1
    
    if num_positive_predicted == 0:
        precision = 1
    else:
        precision = score / num_positive_predicted  # the fraction of predicted “Yes” responses that are correct
    if num_positive_actual == 0:
        recall = 1
    else:
        recall = score / num_positive_actual  # the fraction of “Yes” responses that are predicted correctly

    return precision, recall

# rule-based
print(calc_precision_recall(np.concatenate((inferred_train_bb, inferred_test_bb)), 
                            np.concatenate((np.ones(len(inferred_train_bb)), np.zeros(len(inferred_test_bb))))))

(0.7804878048780488, 0.7239819004524887)
